In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
# from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import random
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [28]:
pergrid_base_path = '../input/pergrid-base/pergrid_base.csv'
pergrid_base_df = pd.read_csv(pergrid_base_path)

In [29]:
pergrid_base_df.head()

In [20]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [31]:
imputer = KNNImputer(n_neighbors=5)
pergrid_base_pred_filled = imputer.fit_transform(pergrid_base_df)
pergrid_base_df = pd.DataFrame(pergrid_base_pred_filled)

# standarize predictors
ss = StandardScaler(with_mean=False, with_std=False)
X_std = ss.fit_transform(pergrid_base_df)

In [32]:
var= ['grid_id',  'aet',  'ai',  'art',  'ewd',  'fa',  'map',  'mat',  'mpdq',  'mtcq',  'pet',  'psn',  'ra',  'rmap',  'rmat',  'tsn',  'mfdf',  'alt',  'shg',  'mtwq',  'tsr',  'wa']
pergrid_base_df2=pd.DataFrame(data=X_std[0:,0:],
                index=pergrid_base_df.index,
                columns=var)

In [34]:
pergrid_base_df=pergrid_base_df2

In [35]:
feature_list = ['aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt', 'shg',
       'mtwq', 'wa']
variable_list = ['aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt', 'shg',
       'mtwq', 'wa', 'tsr']

In [36]:
train, test = train_test_split(pergrid_base_df[variable_list], test_size=0.2, random_state=0)

train.reset_index(inplace=True)
train.drop(['index'], axis=1, inplace=True)

test.reset_index(inplace=True)
test.drop(['index'], axis=1, inplace=True)

In [37]:
train.shape

In [38]:
test.shape

In [39]:
# Combined dataframe containing numerical features only
df = pergrid_base_df[variable_list]
columns = variable_list

# Calculating required amount of rows to display all feature plots
cols = 3
rows = len(columns) // cols + 1

fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(16,30), sharex=False)
fig.set_facecolor("w")

# Adding some distance between plots
plt.subplots_adjust(hspace = 0.3)

# Plots counter
i=0
for r in np.arange(0, rows, 1):
    for c in np.arange(0, cols, 1):
        if i >= len(columns): # If there is no more data columns to make plots from
            axs[r, c].set_visible(False) # Hiding axes so there will be clean background
        else:
            # Train data histogram
            hist1 = axs[r, c].hist(train[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="deepskyblue",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Train Dataset")
            # Test data histogram
            hist2 = axs[r, c].hist(test[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="palevioletred",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Test Dataset")
            axs[r, c].set_title(columns[i], fontsize=14, pad=5)
            axs[r, c].tick_params(axis="y", labelsize=13)
            axs[r, c].tick_params(axis="x", labelsize=13)
            axs[r, c].grid(axis="y")
            axs[r, c].legend(fontsize=13)
                                  
        i+=1
# plt.suptitle("Numerical feature values distribution in both datasets", y=0.99)
plt.savefig("./env_train_test_data_split.png",dpi=300, bbox_inches='tight')
plt.show();

In [40]:
X = train.drop(["tsr"], axis=1)
y = train["tsr"]

# Setting up fold parameters
splits = 10
skf = KFold(n_splits=splits, shuffle=True, random_state=42)

# Creating an array of zeros for storing "out of fold" predictions
oof_preds = np.zeros((X.shape[0],))
preds = 0
model_fi = 0

total_valid_rmse = total_valid_r2 = 0

In [42]:
# Model hyperparameters
rf_params = {'n_estimators': 396,
             'min_samples_split': 10,
             'min_samples_leaf': 5,
             'max_features': 'sqrt',
             'max_depth': 68,
             'criterion': 'mse',
             'bootstrap': True}

In [43]:
# Generating folds and making training and prediction for each of 10 folds
for num, (train_idx, valid_idx) in enumerate(skf.split(X)):
    X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    
    model = RandomForestRegressor(**rf_params) # tree_method='hist'
    model.fit(X_train, y_train)
    
#     # Getting mean test data predictions (i.e. devided by number of splits)
#     preds += model.predict(X_valid) / splits
    
    # Getting mean feature importances (i.e. devided by number of splits) “gain”, “weight”, “cover”, “total_gain” or “total_cover”.
    model_fi += model.feature_importances_ / splits
    
    # Getting validation data predictions. Each fold model makes predictions on an unseen data.
    # So in the end it will be completely filled with unseen data predictions.
    # It will be used to evaluate hyperparameters performance only.
    oof_preds[valid_idx] = model.predict(X_valid)
    oof_preds[train_idx] = model.predict(X_train)
    
    # Getting score for a fold model
    fold_valid_rmse = np.sqrt(mean_squared_error(y_valid, oof_preds[valid_idx]))
    
    # Getting mean score of all fold models (i.e. devided by number of splits)
    total_valid_rmse += fold_valid_rmse / splits
    
print(f"\nOverall RMSE: {total_valid_rmse}")

In [44]:
y_train_pred = model.predict(train.drop(["tsr"], axis=1))
y_train = train['tsr']
train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train_pred, y_train)

print(f'train MAE: {train_mae}; train R2: {train_r2}')

In [45]:
y_test_pred = model.predict(test.drop(["tsr"], axis=1))
y_test = test['tsr']
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test_pred, y_test)

print(f'test MAE: {test_mae}; test R2: {test_r2}')

In [46]:
import shap
import matplotlib.pyplot as plt

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(X)

shap.plots.bar(shap_values)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
shap_interaction_values = explainer.shap_interaction_values(X)

In [ ]:
fig, _ = plt.subplots(figsize=(20, 10))
fig.set_facecolor("w")
shap.plots.beeswarm(shap_values, show=False)

plt.savefig("./summary.png",dpi=300, bbox_inches='tight')

In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.initjs()
shap.plots.force(shap_values[0])

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(test.drop(["tsr"], axis=1))

In [ ]:
fig, _ = plt.subplots(figsize=(20, 10))
fig.set_facecolor("w")
shap.summary_plot(shap_values, test.drop(["tsr"], axis=1), show=False, max_display=10, cmap='viridis')

plt.savefig("./shap_importance_plot.png",dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
shap.plots.scatter(shap_values[:,"fa"], color=shap_values, show=False)
plt.savefig("./fa_plot.png",dpi=300, bbox_inches='tight')

In [ ]:
shap.dependence_plot(
    ("fa", "mpdq"),
    shap_interaction_values, X, show=False
)
plt.savefig("./fa_mpdq_dep.png",dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
shap.plots.scatter(shap_values[:,"ai"], color=shap_values, show=False)
plt.savefig("./ai_plot.png",dpi=300, bbox_inches='tight')

In [ ]:
shap.dependence_plot(
    ("ai", "fa"),
    shap_interaction_values, X, show=False
)
plt.savefig("./ai_fa_dep.png",dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
shap.plots.scatter(shap_values[:,"alt"], color=shap_values, show=False)
plt.savefig("./alt_plot.png",dpi=300, bbox_inches='tight')

In [ ]:
shap.dependence_plot(
    ("alt", "rmap"),
    shap_interaction_values, X, show=False
)
plt.savefig("./alt_rmap_dep.png",dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
shap.plots.scatter(shap_values[:,"mpdq"], color=shap_values, show=False)
plt.savefig("./mpdq_plot.png",dpi=300, bbox_inches='tight')

In [ ]:
shap.dependence_plot(
    ("mpdq", "art"),
    shap_interaction_values, X, show=False
)
plt.savefig("./mpdq_art.png",dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
shap.plots.scatter(shap_values[:,"mat"], color=shap_values, show=False)
plt.savefig("./mat_plot.png",dpi=300, bbox_inches='tight')

In [ ]:
shap.dependence_plot(
    ("mat", "mpdq"),
    shap_interaction_values, X, show=False
)
plt.savefig("./mat_mpdq.png",dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
shap.plots.scatter(shap_values[:,"map"], color=shap_values, show=False)
plt.savefig("./map_plot.png",dpi=300, bbox_inches='tight')

In [ ]:
shap.plots.bar(explainer(test.drop(["tsr"], axis=1)))

In [ ]:
# columns = test.drop(["tsr"], axis=1).columns

# # adjust nrows, ncols to fit all your columns
# fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(30, 30))
# fig.set_facecolor("w")

# axes = axes.ravel()

# for i, col in enumerate(columns):
#     shap.dependence_plot(col, shap_values, test.drop(["tsr"], axis=1), ax=axes[i], show=False)
    
# plt.savefig("./partial_dependence_plot.png",
#             dpi=300, bbox_inches='tight',
#            facecolor=fig.get_facecolor(), edgecolor='none')
# plt.show();

In [ ]:
# shap.initjs()
# shap.force_plot(explainer.expected_value, shap_values[1], feature_names=test.columns.tolist()[:-1])